In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# from pandas import read_csv
# import numpy as np
# dataframe = read_csv('/content/drive/MyDrive/Privacy/creditcard.csv', header=None)
# dataframe.head()

# target_variable = 30
# X_input = dataframe.drop([target_variable], axis=1).values
# y_input = dataframe[target_variable].values

# indx = []
# for i in range(len(y_input)): 
#     if y_input[i] == 0:
#         indx.append(i)

# X_input =  X_input[indx]
# y_input = y_input[indx]

from pandas import read_csv
import numpy as np
dataframe = read_csv('/content/drive/MyDrive/Privacy/creditcard.csv', header=None)
dataframe.head()
from scipy.special import expit
target_variable = 30
X_train = dataframe.drop([target_variable], axis=1).values
Y_train = dataframe[target_variable].values


import pandas as pd
from pandas import read_csv
import numpy as np

dataframe = read_csv('/content/drive/MyDrive/Privacy/creditcard.csv', header=None)
dataframe.head()
target_variable = 30

index_p = np.where(Y_train==1)
index_n = np.where(Y_train==0)
# print(index_p[0].shape)

positive_data_real = dataframe.loc[index_p[0] , : ][:480]
# print(positive_data_real.shape)

negative_data_real = dataframe.loc[index_n[0] , : ]
negative_data_real = negative_data_real[:1500]
# print(negative_data_real.shape)

real_data = pd.concat([positive_data_real, negative_data_real])
X_input = real_data.drop([target_variable], axis=1).values
y_input = real_data[target_variable].values

print(X_input.shape)
print(y_input.shape)

from sklearn.model_selection import train_test_split
X_input, X_test, y_input, y_test = train_test_split(X_input, y_input, test_size = 0.0001, random_state = 42)
# print(X_train.shape)
print(X_test.shape)
print(X_input.shape)
print(y_input.shape)
X_input = expit(X_input)


(1980, 30)
(1980,)
(1, 30)
(1979, 30)
(1979,)


# New data

In [24]:
from pandas import read_csv
from scipy.special import expit
from sklearn.utils import shuffle
import pickle
def load_dataset(full_path = '/content/drive/MyDrive/Privacy/real_data_train.csv'):
  # load the dataset as a numpy array
  file = open(full_path,'rb')
  data = pickle.load(file)
  file.close()
  # data = pickle.load(full_path)
  # retrieve numpy array
  data = data.values
  # split into input and output elements
  X, y = data[:, :-1], data[:, -1]
  return X, y

X, y = load_dataset()
X = expit(X)
X_input, y_input = shuffle(X, y)
print(X_input,y_input)

[[9.96796491e-01 3.91064463e-01 5.93909281e-01 7.98396107e-01]
 [8.60458588e-02 4.73639464e-01 8.73647350e-01 7.56907680e-01]
 [1.11390145e-02 1.69318076e-06 9.99999970e-01 3.84421580e-02]
 ...
 [2.37045093e-02 2.01624874e-04 9.99995726e-01 2.77844276e-01]
 [7.72889604e-01 9.99845922e-01 9.85447182e-02 3.08643227e-01]
 [8.05498190e-02 9.61312478e-05 9.99875639e-01 3.77538319e-01]] [0. 1. 1. 0. 1. 1. 0. 1. 1. 0. 1. 0. 0. 1. 1. 0. 0. 0. 0. 0. 1. 0. 0. 1.
 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 1. 0. 1. 0. 0. 0. 1. 0. 1. 1. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 1. 0. 0. 1. 1. 0. 0. 1. 0. 0. 1. 0. 1.
 0. 1. 1. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 1. 1. 0.
 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0.
 1. 1. 1. 0. 0. 0. 1. 1. 1. 0. 0. 1. 0. 0. 0. 1. 1. 1. 1. 0. 1. 0. 0. 1.
 0. 1. 0. 1. 0. 1. 1. 0. 1. 1. 1. 0. 1. 1. 0. 0. 0. 1. 1. 0. 0. 1. 0. 0.
 0. 0. 0. 1. 1. 1. 1. 1. 1. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 1.
 1. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0

In [25]:
'''
Code inspired from: 
https://github.com/greenelab/SPRINT_gan/blob/master/ac_gan.py
'''

from __future__ import print_function

from collections import defaultdict
try:
    import cPickle as pickle
except ImportError:
    import pickle
from PIL import Image

from six.moves import range

import tensorflow as tf
import keras.backend as K
from keras.datasets import mnist
from keras import layers
from keras.layers import Input, Dense, Reshape, Flatten, Embedding, Dropout
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras.utils.generic_utils import Progbar
import numpy as np
import os
import argparse
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
training_size = 999
# training_size = 400
np.random.seed(0)
K.set_image_data_format('channels_first')

def build_generator(latent_size):
    # we will map a pair of (z, L), where z is a latent vector and L is a
    # label drawn from P_c, to image space (..., 1, 28, 28)
    
    print('Generator')
    cnn = Sequential()

    cnn.add(Dense(2*latent_size, input_dim=latent_size, activation='relu'))
    cnn.add(Dense(10, activation='relu'))
    cnn.add(Dense(4, activation='relu'))

  
    cnn.summary()

    # this is the z space commonly refered to in GAN papers
    latent = Input(shape=(latent_size, ))

    # this will be our label
    patient_class = Input(shape=(1,), dtype='int32')

    # 10 classes in MNIST
    cls = Flatten()(Embedding(
                        2, latent_size,
                        embeddings_initializer='glorot_normal')(patient_class))

    # hadamard product between z-space and a class conditional embedding
    h = layers.multiply([latent, cls])

    fake_patient = cnn(h)

    return Model([latent, patient_class], fake_patient)


def build_discriminator():
    # build a relatively standard conv net, with LeakyReLUs as suggested in
    # the reference paper
    print('Discriminator')
    cnn = Sequential()
    cnn.add(Dense(8, activation='relu', input_shape=(4,)))
    cnn.add(Dense(10, activation='relu'))
    cnn.add(Dense(4, activation='relu'))
    # cnn.add(Dropout(0.3))
    cnn.add(Dense(1, activation='relu'))

    patient = Input(shape=(4,))

    features = cnn(patient)
    cnn.summary()

    fake = Dense(1, activation='sigmoid', name='generation')(features)
    # aux could probably be 1 sigmoid too...
    aux = Dense(2, activation='softmax', name='auxiliary')(features)

    return Model(patient, [fake, aux])



epochs = 400
batch_size = 32
latent_size = 2

# Adam parameters suggested in https://arxiv.org/abs/1511.06434
adam_lr = 0.0002
adam_beta_1 = 0.5

directory = ('/content/drive/MyDrive/Privacy/sprint/acgan' + '_' + str(epochs) + '_' +
              str(adam_lr) + '_' + str(batch_size) + '/')

if not os.path.exists(directory):
    os.mkdir(directory)

discriminator = build_discriminator()
discriminator.compile(
    optimizer=Adam(lr=adam_lr, beta_1=adam_beta_1),
    loss=['binary_crossentropy', 'sparse_categorical_crossentropy']
)

# build the generator
generator = build_generator(latent_size)
generator.compile(optimizer=Adam(lr=adam_lr, beta_1=adam_beta_1),
                  loss='binary_crossentropy')

latent = Input(shape=(latent_size, ))
image_class = Input(shape=(1,), dtype='int32')

# get a fake image
fake = generator([latent, image_class])

# we only want to be able to train generation for the combined model
discriminator.trainable = False
fake, aux = discriminator(fake)
combined = Model([latent, image_class], [fake, aux])

combined.compile(
    optimizer=Adam(lr=adam_lr, beta_1=adam_beta_1),
    loss=['binary_crossentropy', 'sparse_categorical_crossentropy']
)


X_input 
print(X_input.shape, y_input.shape)

X_train = X_input[:training_size]
X_test = X_train

y_train = y_input[:training_size]
y_test = y_train

num_train, num_test = X_train.shape[0], X_test.shape[0]
print(num_train, num_test)
train_history = defaultdict(list)
test_history = defaultdict(list)
privacy_history = []

with tf.Session() as sess:
    eps = tf.placeholder(tf.float32)
    delta = tf.placeholder(tf.float32)

    for epoch in range(epochs):
        print('Epoch {} of {}'.format(epoch + 1, epochs))

        num_batches = int(X_train.shape[0] / batch_size)
        progress_bar = Progbar(target=num_batches)

        epoch_gen_loss = []
        epoch_disc_loss = []

        for index in range(num_batches):
            progress_bar.update(index)
            # generate a new batch of noise
            noise = np.random.uniform(-1, 1, (batch_size, latent_size))

            # get a batch of real images
            image_batch = X_train[index * batch_size:(index + 1) * batch_size]
            label_batch = y_train[index * batch_size:(index + 1) * batch_size]

            # sample some labels from p_c
            sampled_labels = np.random.randint(0, 2, batch_size)       
            generated_images = generator.predict(
                [noise, sampled_labels.reshape((-1, 1))], verbose=0)

            print(image_batch.shape)
            print(generated_images.shape)
            X = np.concatenate((image_batch, generated_images))
            y = np.array([1] * batch_size + [0] * batch_size)
            aux_y = np.concatenate((label_batch, sampled_labels), axis=0)

            epoch_disc_loss.append(discriminator.train_on_batch(
                X, [y, aux_y]))


            # make new noise. we generate 2 * batch size here such that we have
            # the generator optimize over an identical number of images as the
            # discriminator
            noise = np.random.uniform(-1, 1, (2 * batch_size, latent_size))
            sampled_labels = np.random.randint(0, 2, 2 * batch_size)                            

            trick = np.ones(2 * batch_size)

            epoch_gen_loss.append(combined.train_on_batch(
                [noise, sampled_labels.reshape((-1, 1))],
                [trick, sampled_labels]))

        print('\nTesting for epoch {}:'.format(epoch + 1))

        # evaluate the testing loss here
      
        #### start of loss calculation
        # generate a new batch of noise
        noise = np.random.uniform(-1, 1, (num_test, latent_size))

        # sample some labels from p_c and generate images from them
        sampled_labels = np.random.randint(0, 2, num_test)
      
        generated_images = generator.predict(
            [noise, sampled_labels.reshape((-1, 1))], verbose=False)


        X = np.concatenate((X_test, generated_images))
        y = np.array([1] * num_test + [0] * num_test)
        aux_y = np.concatenate((y_test, sampled_labels), axis=0)

        # see if the discriminator can figure itself out...
        discriminator_test_loss = discriminator.evaluate(
            X, [y, aux_y], verbose=False)

        discriminator_train_loss = np.mean(np.array(epoch_disc_loss), axis=0)

        # make new noise
        noise = np.random.uniform(-1, 1, (2 * num_test, latent_size))
        sampled_labels = np.random.randint(0, 2, 2 * num_test)

        trick = np.ones(2 * num_test)

        generator_test_loss = combined.evaluate(
            [noise, sampled_labels.reshape((-1, 1))],
            [trick, sampled_labels], verbose=False)

        generator_train_loss = np.mean(np.array(epoch_gen_loss), axis=0)

        #### end of loss calculation

    # generate data
    if epoch == (epochs-1):
      noise = np.random.uniform(-1, 1, (24807 , latent_size))
      

      sampled_labels = np.random.randint(0, 2, 284807)                                       
      # sampled_labels = np.random.choice(np.arange(0, 2), p=[0.9,0.1], 284807)
      
      generated_data = generator.predict([noise, sampled_labels.reshape((-1, 1))], verbose=False)
      print(sampled_labels.shape)
      print(generated_data.shape)
      print("finished")

    

Discriminator
Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_18 (Dense)             (None, 8)                 40        
_________________________________________________________________
dense_19 (Dense)             (None, 10)                90        
_________________________________________________________________
dense_20 (Dense)             (None, 4)                 44        
_________________________________________________________________
dense_21 (Dense)             (None, 1)                 5         
Total params: 179
Trainable params: 179
Non-trainable params: 0
_________________________________________________________________
Generator
Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_22 (Dense)             (None, 4)                 12        
__________

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


Streaming output truncated to the last 5000 lines.
(32, 4)
 4/31 [==>...........................] - ETA: 0s(32, 4)
(32, 4)
(32, 4)
(32, 4)
(32, 4)
(32, 4)
(32, 4)
(32, 4)
 8/31 [======>.......................] - ETA: 0s(32, 4)
(32, 4)
(32, 4)
(32, 4)
(32, 4)
(32, 4)
11/31 [=========>....................] - ETA: 0s(32, 4)
(32, 4)
(32, 4)
(32, 4)
(32, 4)
(32, 4)
(32, 4)
(32, 4)
15/31 [=============>................] - ETA: 0s(32, 4)
(32, 4)
(32, 4)
(32, 4)
(32, 4)
(32, 4)
(32, 4)
(32, 4)
19/31 [=================>............] - ETA: 0s(32, 4)
(32, 4)
(32, 4)
(32, 4)
(32, 4)
(32, 4)
(32, 4)
(32, 4)
23/31 [=====================>........] - ETA: 0s(32, 4)
(32, 4)
(32, 4)
(32, 4)
(32, 4)
(32, 4)
(32, 4)
(32, 4)
27/31 [=========================>....] - ETA: 0s(32, 4)
(32, 4)
(32, 4)
(32, 4)
(32, 4)
(32, 4)
(32, 4)
(32, 4)

Testing for epoch 324:
Epoch 325 of 400
 0/31 [..............................] - ETA: 0s(32, 4)
(32, 4)
(32, 4)
(32, 4)
(32, 4)
(32, 4)
(32, 4)
(32, 4)
 4/31 [==>..........

In [26]:
import numpy as np
import pandas as pd 
print(generated_data.shape)
print(sampled_labels.shape)




# print(np.where(sampled_labels==1))
pos_labels = []
for item in sampled_labels:
  if int(item)== 1:
    pos_labels.append(item)

pos_labels = np.array(pos_labels)
print(pos_labels.shape)

number_pos = 0
number_neg = 0
gendata1000 = []
label1000 = []
for i in range(len(sampled_labels)):
      if sampled_labels[i] == 1 and number_pos<3000:
          gendata1000.append(generated_data[i])
          label1000.append(sampled_labels[i])
          number_pos += 1
      if sampled_labels[i] == 0 and number_neg<3000:
          gendata1000.append(generated_data[i])
          label1000.append(sampled_labels[i])
          number_neg += 1
      if number_neg>3000 and number_pos>3000:
          break

# #save data
# proto_tensor = tf.make_tensor_proto(gendata1000)  # convert tensor a to a proto tensor
# data_gen = tf.make_ndarray(proto_tensor) #convert tf tesor to np array
data_gen = np.array(gendata1000)
print(data_gen.shape)
pd.DataFrame(data_gen).to_csv("/content/drive/MyDrive/Privacy/sprint/SPRINT_data_x_new4.csv")

# #save labels
proto_tensor = tf.make_tensor_proto(label1000)  # convert tensor a to a proto tensor
label_gen = tf.make_ndarray(proto_tensor) #convert tf tesor to np array
print(label_gen.shape)
pd.DataFrame(label_gen).to_csv("/content/drive/MyDrive/Privacy/sprint/SPRINT_data_y_new4.csv")


# # save positive labels
# proto_tensor = tf.make_tensor_proto(pos_labels)  # convert tensor a to a proto tensor
# pos_gen = tf.make_ndarray(proto_tensor) #convert tf tesor to np array
# print(pos_gen.shape)

# pd.DataFrame(pos_gen).to_csv("/content/drive/MyDrive/Privacy/sprint/SPRINT_positive_labels_new.csv")

(24807, 4)
(284807,)
(142514,)
(6000, 4)
(6000,)


In [27]:
# load the dataset
from pandas import read_csv
from scipy.special import expit
from sklearn.utils import shuffle
import pickle
def load_dataset_train(full_path = '/content/drive/MyDrive/Privacy/real_data_train.csv'):
  # load the dataset as a numpy array
  with open(full_path, 'rb') as f:
    data = pickle.load(f)
  # retrieve numpy array
  data = data.values
  # split into input and output elements
  X, y = data[:, :-1], data[:, -1]
  return X, y

X_train, y_train = load_dataset_train()
X_train = expit(X_train)
print(X_train.shape, y_train.shape)
print(X_train[0])


def load_dataset_test(full_path = '/content/drive/MyDrive/Privacy/real_data_test.csv'):
  # load the dataset as a numpy array
  with open(full_path, 'rb') as f:
    data = pickle.load(f)
  # retrieve numpy array
  data = data.values
  # split into input and output elements
  X, y = data[:, :-1], data[:, -1]
  return X, y

X_test, y_test = load_dataset_test()
X_test = expit(X_test)
print(X_test.shape)

(1000, 4) (1000,)
[0.99411941 0.99982842 0.05258531 0.18114637]
(372, 4)


In [28]:
from pandas import read_csv
# import x data
dataframe = read_csv('/content/drive/MyDrive/Privacy/sprint/SPRINT_data_x_new4.csv', header=None)
x_sprint = dataframe.values
X_sprint = x_sprint[:,1:]
print(X_sprint.shape)

# import y data
dataframe = read_csv('/content/drive/MyDrive/Privacy/sprint/SPRINT_data_y_new4.csv', header=None)
Y_sprint = dataframe.values[:,1]
print(Y_sprint.shape)


print(X_sprint.shape) #this is a np array with 450 (it is max) positive and 1500 negative


print(Y_sprint.shape)
print(Y_sprint[1])


(6001, 4)
(6001,)
(6001, 4)
(6001,)
1.0


In [29]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras import backend as K
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve
import tensorflow as tf


def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


model = Sequential([
Dense(input_dim = 4, units = 2, activation = 'relu'),
# Dense(units = 32, activation = 'relu'),
# Dropout(0.2),
Dense(units = 10, activation = 'relu'),
Dense(units = 4, activation = 'relu'),
Dense(units =1, activation = 'sigmoid'),])


In [30]:
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy', f1_m, precision_m, recall_m])
model.fit(X_sprint, Y_sprint, batch_size = 10, epochs = 5)
# evaluate the model
loss, accuracy, f1_score, precision, recall = model.evaluate(X_test, y_test, verbose=0)
print(loss, accuracy, f1_score, precision, recall)

Train on 6001 samples
Epoch 1/10
6001/6001 [==============================] - 3s 461us/sample - loss: 0.6674 - acc: 0.5344 - f1_m: 0.5704 - precision_m: 0.4918 - recall_m: 0.7631
Epoch 2/10
6001/6001 [==============================] - 3s 469us/sample - loss: 0.5894 - acc: 0.6437 - f1_m: 0.6793 - precision_m: 0.5947 - recall_m: 0.8784
Epoch 3/10
6001/6001 [==============================] - 3s 460us/sample - loss: 0.5299 - acc: 0.7075 - f1_m: 0.7285 - precision_m: 0.6385 - recall_m: 0.9222
Epoch 4/10
6001/6001 [==============================] - 3s 464us/sample - loss: 0.5046 - acc: 0.7330 - f1_m: 0.7503 - precision_m: 0.6529 - recall_m: 0.9513
Epoch 5/10
6001/6001 [==============================] - 3s 466us/sample - loss: 0.4988 - acc: 0.7350 - f1_m: 0.7532 - precision_m: 0.6567 - recall_m: 0.9549
Epoch 6/10
6001/6001 [==============================] - 3s 441us/sample - loss: 0.4971 - acc: 0.7369 - f1_m: 0.7573 - precision_m: 0.6594 - recall_m: 0.9588
Epoch 7/10
6001/6001 [==============

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '
